## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gmaps
import requests

# Import the API key.
from config import g_key

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Meulaboh,ID,4.1363,96.1285,83.48,76,70,3.89,broken clouds
1,1,Khatanga,RU,71.9667,102.5000,-6.86,96,100,6.67,overcast clouds
2,2,Kodiak,US,57.7900,-152.4072,26.11,74,100,19.57,light snow
3,3,Punta Arenas,CL,-53.1500,-70.9167,59.11,44,100,5.99,overcast clouds
4,4,Namibe,AO,-15.1961,12.1522,70.86,83,100,6.26,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 25
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Meulaboh,ID,4.1363,96.1285,83.48,76,70,3.89,broken clouds
2,2,Kodiak,US,57.7900,-152.4072,26.11,74,100,19.57,light snow
3,3,Punta Arenas,CL,-53.1500,-70.9167,59.11,44,100,5.99,overcast clouds
4,4,Namibe,AO,-15.1961,12.1522,70.86,83,100,6.26,overcast clouds
5,5,Saint-Joseph,RE,-21.3667,55.6167,79.11,71,0,7.00,clear sky
6,6,Sezemice,CZ,50.0665,15.8527,36.41,67,0,3.00,clear sky
7,7,Hilo,US,19.7297,-155.0900,74.25,86,4,3.00,clear sky
8,8,Bluff,NZ,-46.6000,168.3333,67.80,63,91,8.05,overcast clouds
9,9,Asosa,ET,10.0667,34.5333,58.93,49,0,6.20,clear sky
10,10,Santa Barbara,US,34.7333,-120.0343,54.41,47,0,4.61,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                531
City                   531
Country                523
Lat                    531
Lng                    531
Max Temp               531
Humidity               531
Cloudiness             531
Wind Speed             531
Current Description    531
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                523
City                   523
Country                523
Lat                    523
Lng                    523
Max Temp               523
Humidity               523
Cloudiness             523
Wind Speed             523
Current Description    523
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Meulaboh,ID,83.48,broken clouds,4.1363,96.1285,
2,Kodiak,US,26.11,light snow,57.7900,-152.4072,
3,Punta Arenas,CL,59.11,overcast clouds,-53.1500,-70.9167,
4,Namibe,AO,70.86,overcast clouds,-15.1961,12.1522,
5,Saint-Joseph,RE,79.11,clear sky,-21.3667,55.6167,
6,Sezemice,CZ,36.41,clear sky,50.0665,15.8527,
7,Hilo,US,74.25,clear sky,19.7297,-155.0900,
8,Bluff,NZ,67.80,overcast clouds,-46.6000,168.3333,
9,Asosa,ET,58.93,clear sky,10.0667,34.5333,
10,Santa Barbara,US,54.41,clear sky,34.7333,-120.0343,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

 # 6c. Get latitude and longitude from DataFrame
        lat = row["Lat"]
        lng = row["Lng"]
        
        # Add the latitude and longitude to location key for the params dictionary.
        params["location"] = f"{lat},{lng}"
 
        # 6d. Set up the base URL for the Google Directions API to get JSON data.
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # 6e. Make request and retrieve the JSON data from the search. 
        hotels = requests.get(base_url, params=params).json() 
    
 # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")             

In [9]:
# replace any empty strings in the Hotel Name column with np.nan
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Meulaboh,ID,83.48,broken clouds,4.1363,96.1285,NaN
2,Kodiak,US,26.11,light snow,57.7900,-152.4072,NaN
3,Punta Arenas,CL,59.11,overcast clouds,-53.1500,-70.9167,NaN
4,Namibe,AO,70.86,overcast clouds,-15.1961,12.1522,NaN
5,Saint-Joseph,RE,79.11,clear sky,-21.3667,55.6167,NaN
...,...,...,...,...,...,...,...
682,Ajdabiya,LY,54.43,scattered clouds,30.7554,20.2263,NaN
683,Manyana,BW,68.38,few clouds,-23.4000,21.7167,NaN
684,Vanimo,PG,84.27,overcast clouds,-2.6741,141.3028,NaN
685,Utiel,ES,37.65,clear sky,39.5667,-1.2000,NaN


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
686,Nurota,UZ,32.09,overcast clouds,40.5614,65.6886,Guest House Ruslan Nurata & Tours


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation_database.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
cur_desc = hotel_df["Current Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))